In [3]:
from ortools.linear_solver import pywraplp

def optimize_production():
    # Create the solver
    solver = pywraplp.Solver.CreateSolver('SCIP')

    # Constants
    M = 100  # Big M for enforcing conditional constraints

    # Problem parameters
    weeks = 8
    max_p = 100
    max_h = 80
    initial_inventory_p = 125
    initial_inventory_h = 143
    production_cost_p = 225
    production_cost_h = 310
    inventory_cost = 0.195 / 52  # weekly cost per unit
    changeover_cost = 500

    # Weekly demand for each product type
    demand_p = [55, 44, 0, 45, 45, 36, 35, 35]
    demand_h = [38, 30, 0, 48, 48, 58, 57, 58]

    # Decision variables
    # Production and inventory for each week
    production_p = [solver.IntVar(0, max_p, f'Week_{i+1}_P_Production') for i in range(weeks)]
    production_h = [solver.IntVar(0, max_h, f'Week_{i+1}_H_Production') for i in range(weeks)]
    inventory_p = [solver.IntVar(0, solver.infinity(), f'Week_{i+1}_P_Inventory') for i in range(weeks)]
    inventory_h = [solver.IntVar(0, solver.infinity(), f'Week_{i+1}_H_Inventory') for i in range(weeks)]
    is_producing_p = [solver.BoolVar(f'is_producing_p_{i}') for i in range(weeks)]
    is_producing_h = [solver.BoolVar(f'is_producing_h_{i}') for i in range(weeks)]
    changeover = [solver.BoolVar(f'Changeover_{i}') for i in range(1, weeks)]

    # Initial inventory balance
    solver.Add(inventory_p[0] == initial_inventory_p - demand_p[0] + production_p[0])
    solver.Add(inventory_h[0] == initial_inventory_h - demand_h[0] + production_h[0])

    # Constraints for all weeks
    for i in range(weeks):
        # Production indicates production mode
        solver.Add(production_p[i] <= M * is_producing_p[i])
        solver.Add(production_h[i] <= M * is_producing_h[i])
        solver.Add(production_p[i] >= is_producing_p[i])
        solver.Add(production_h[i] >= is_producing_h[i])

        # Only one type of head can be produced
        solver.Add(is_producing_p[i] + is_producing_h[i] == 1)

        # Inventory equations
        if i > 0:
            solver.Add(inventory_p[i] == inventory_p[i-1] + production_p[i] - demand_p[i])
            solver.Add(inventory_h[i] == inventory_h[i-1] + production_h[i] - demand_h[i])

            # Safety stock requirements
            if i < weeks - 1:
                solver.Add(inventory_p[i] >= 0.8 * demand_p[i + 1])
                solver.Add(inventory_h[i] >= 0.8 * demand_h[i + 1])
            
            # Changeover constraints
            solver.Add(changeover[i-1] >= is_producing_p[i-1] + is_producing_h[i] - 1)
            solver.Add(changeover[i-1] >= is_producing_h[i-1] + is_producing_p[i] - 1)

            # Non-negativity constraints for production
            solver.Add(production_p[i] >= 0)
            solver.Add(production_h[i] >= 0)

    # Objective function
    total_production_cost = solver.Sum(production_p[i] * production_cost_p + production_h[i] * production_cost_h for i in range(weeks))
    total_changeover_cost = solver.Sum(changeover[i-1] * changeover_cost for i in range(1, weeks))
    total_inventory_cost = solver.Sum(inventory_p[i] * inventory_cost + inventory_h[i] * inventory_cost for i in range(weeks))
    total_cost = total_production_cost + total_changeover_cost + total_inventory_cost

    solver.Minimize(total_cost)

    # Solve the model
    status = solver.Solve()

    # Print results
    if status == pywraplp.Solver.OPTIMAL:
        print("Solution:")
        print("Objective value =", round(solver.Objective().Value(), 2))
        print("Total production cost =", total_production_cost.solution_value())
        print("Total changeover cost =", total_changeover_cost.solution_value())
        print("Total inventory holding cost =", round(total_inventory_cost.solution_value(), 2))
        for i in range(weeks):
            print(f"Week {i+1} Production P = ", production_p[i].solution_value())
            print(f"Week {i+1} Production H =", production_h[i].solution_value())
            print(f"Week {i+1} Inventory P =", inventory_p[i].solution_value())
            print(f"Week {i+1} Inventory H =", inventory_h[i].solution_value())
    else:
        print("The problem does not have an optimal solution.")

# Call the function
optimize_production()


Solution:
Objective value = 98894.35
Total production cost = 98390.0
Total changeover cost = 500.0
Total inventory holding cost = 4.35
Week 1 Production P =  1.0
Week 1 Production H = 0.0
Week 1 Inventory P = 71.0
Week 1 Inventory H = 105.0
Week 2 Production P =  69.0
Week 2 Production H = 0.0
Week 2 Inventory P = 96.0
Week 2 Inventory H = 75.0
Week 3 Production P =  100.0
Week 3 Production H = 0.0
Week 3 Inventory P = 196.0
Week 3 Inventory H = 75.0
Week 4 Production P =  0.0
Week 4 Production H = 12.0
Week 4 Inventory P = 151.0
Week 4 Inventory H = 39.0
Week 5 Production P =  0.0
Week 5 Production H = 56.0
Week 5 Inventory P = 106.0
Week 5 Inventory H = 47.0
Week 6 Production P =  0.0
Week 6 Production H = 57.0
Week 6 Inventory P = 70.0
Week 6 Inventory H = 46.0
Week 7 Production P =  0.0
Week 7 Production H = 58.0
Week 7 Inventory P = 35.0
Week 7 Inventory H = 47.0
Week 8 Production P =  0.0
Week 8 Production H = 11.0
Week 8 Inventory P = 0.0
Week 8 Inventory H = 0.0
